# Notebook for trying to recreate the delta->omikron wave of fall/winter 2021

In [40]:
%matplotlib widget

import sys
sys.path.append("./../scripts")
import PandemiXFunctions as pf
import PandemiXModelFunctions as pmf

import os


import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
             
             
from scipy.integrate import cumtrapz
# int_I = cumtrapz(np.exp(u_H*ts)*Is,ts, initial=0)
# Hs = np.exp(-u_H*ts) * (r_chr*pars_full['gamma']*int_I + k) 

In [41]:
# Reload PandemiXModelFunctions, in case changes have been made
import importlib 
importlib.reload(pmf)

<module 'PandemiXModelFunctions' from 'c:\\Users\\rakrpe\\OneDrive - Roskilde Universitet\\Documents\\PandemiX\\GithubRepos\\PandemiX\\modelling\\PandemiXModelFunctions.py'>

In [42]:
path_figs = os.getcwd() + '\\Figures\\'
path_figs = path_figs + 'TwoStrainModel_'

# Start by just getting the delta growth correct

In [43]:
# Initialize everything
iniInf = 0.000005
iniVacc = 0.8 # Approximately DK's vaccinationsrate at the end of summer 2021
iniImmu = 0.05 # Approximation. Should only be "natural immunity"
iniImmu = iniImmu * (1-iniVacc) # For splitting previous infections between vaccinated and susceptible

V_0 = iniVacc 
IS_0 = 0
IV_0 = 0
Y_0 = iniInf
R01_0 = iniImmu
R10_0 = 0
I01_0 = 0
Y10_0 = 0
S_0 = 1 - V_0 - IS_0 - IV_0 - Y_0 - R01_0 - R10_0 - I01_0 - Y10_0 

init = {    
    'S' : S_0,
    'V' : V_0,
    'IS' : IS_0,
    'IV' : IV_0,
    'Y' : Y_0,
    'R01' : R01_0,
    'R10' : R10_0,
    'I01' : I01_0,
    'Y10' : Y10_0,
}

gammas = 1/7
beta_Y = 6*gammas # For zero initial immunity
beta_Y = 6.5*gammas
beta_I = 2*gammas

pars = {
    'beta_IS_S' : beta_I,
    'beta_IV_S' : beta_I,
    'beta_I01_S' : beta_I,
    'beta_IS_V' : beta_I,
    'beta_IV_V' : beta_I,
    'beta_I01_V' : beta_I,
    'beta_IS_R01' : beta_I,
    'beta_IV_R01' : beta_I,
    'beta_I01_R01' : beta_I,
    'beta_Y_S' : beta_Y,
    'beta_Y10_S' : beta_Y,
    'beta_Y_R10' : beta_Y,
    'beta_Y10_R10' : beta_Y,
    'gamma_IS' : gammas,
    'gamma_IV' : gammas,
    'gamma_Y' : gammas,
    'gamma_I01' : gammas,
    'gamma_Y10' : gammas,
    }


In [44]:
tEnd = 500
sch1 = pmf.Scheme('OmikronDeltaFull',init,pars,0,tEnd,Changes=[])

In [45]:
sch1.simulate()

In [46]:
curRes = sch1.result 
RTs = 1 - curRes.y.sum(axis=0)

DKpop = 5800000

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,(ax1,ax2) = plt.subplots(2,1)
ax1.fill_between(curRes.t,curRes.Y*DKpop,color='r',label='Delta')
ax2.fill_between(curRes.t,curRes.Y*DKpop,color='r',label='Delta')

t_width = 30*5
t_init = 90 # Approximately 2021-10-01

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=8000)
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax2.set_xlim([t_init,t_init+t_width])

ax1.set_xlabel('t')
ax2.set_xlabel('t')

ax2.set_title('Total simulated period')
ax2.set_title('Zoom on relevant period')

# ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
# ax1.set_ylim([0,1])
# ax1.set_xlim([curRes.t[0],curRes.t[-1]])
# ax1.set_xlabel('t')
# ax1.set_ylabel('Share of population')
fig.tight_layout()

# fig.savefig(path_figs+'IthenY')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
curRes = sch1.result 
RTs = 1 - curRes.y.sum(axis=0)

fig,ax1 = plt.subplots()
curLabels = [   
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                r'$I_{0,1}$: Infectious'+'\n(previous Y)',
                r'$Y$: Infectious',
                r'$Y_{1,0}$: Infectious'+'\n(previous I)',
                r'$R_{1,0}$: Recovered, I',
                r'$R_{0,1}$: Recovered, Y',
                r'$R_{1,1}$: Immune to both',
                r'$S$: Susceptible',
                r'$V$: Vaccinated'
                ]
curColors = ['xkcd:orange','xkcd:light orange','xkcd:yellow','xkcd:red','xkcd:light red','xkcd:green','xkcd:light green','xkcd:dark green','xkcd:sky blue','xkcd:grey']
ax1.stackplot(curRes.t,
                curRes.IS,
                curRes.IV,
                curRes.I01,
                curRes.Y,
                curRes.Y10,
                curRes.R10,
                curRes.R01,
                RTs,
                curRes.S,
                curRes.V,
                labels=curLabels,colors=curColors)

ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
ax1.set_ylim([0,0.25])
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

# fig.savefig(path_figs+'IthenY')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
tChange = t_init + 50
tChange = t_init + 40
toAdd = iniInf
dictToAdd =  {  
    'S' : -toAdd,
    'V' : 0,
    'IS' : toAdd,
    'IV' : 0,
    'Y' : 0,
    'R01' : 0,
    'R10' : 0,
    'I01' : 0,
    'Y10' : 0,
}
addI = pmf.Change(tChange, AddVariables=dictToAdd)

sch2 = sch1.copy()
sch2.addChange(addI)
sch2.simulate()

In [49]:
curRes = sch2.result 
RTs = 1 - curRes.y.sum(axis=0)

fig,(ax1,ax2) = plt.subplots(2,1)
curLabels = [   
                r'$Y$: Infectious',
                r'$Y_{1,0}$: Infectious'+'\n(previous I)',
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                r'$I_{0,1}$: Infectious'+'\n(previous Y)',
                ]
curColors = ['xkcd:red','xkcd:light red','xkcd:orange','xkcd:light orange','xkcd:yellow']
                
for ax in (ax1,ax2):
    ax.stackplot(curRes.t,
                DKpop*curRes.Y,
                DKpop*curRes.Y10,
                DKpop*curRes.IS,
                DKpop*curRes.IV,
                DKpop*curRes.I01,
                labels=curLabels,colors=curColors)

t_width = 30*5
t_init = 90

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=8000)
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax2.set_xlim([t_init,t_init+t_width])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(90.0, 240.0)

# Try with lockdown mid december (-ish) 

In [59]:
tChange2 = t_init + 75 # Approximately December 15th
betaMult = 0.65
betaMult_Y = betaMult 
betaMult_I = betaMult 
# betaMult = 0.7
# betaMult_Y = betaMult *0.8
# betaMult_I = betaMult 
dictToMult = {
'beta_IS_S' : betaMult_I,
'beta_IV_S' : betaMult_I,
'beta_I01_S' : betaMult_I,
'beta_IS_V' : betaMult_I,
'beta_IV_V' : betaMult_I,
'beta_I01_V' : betaMult_I,
'beta_IS_R01' : betaMult_I,
'beta_IV_R01' : betaMult_I,
'beta_I01_R01' : betaMult_I,
'beta_Y_S' : betaMult_Y,
'beta_Y10_S' : betaMult_Y,
'beta_Y_R10' : betaMult_Y,
'beta_Y10_R10' : betaMult_Y,
'gamma_IS' : 1,
'gamma_IV' : 1,
'gamma_Y' : 1,
'gamma_I01' : 1,
'gamma_Y10' : 1,
}
betaMultChange = pmf.Change(tChange2, MultiplyParameters=dictToMult)

sch3 = sch2.copy()
sch3.addChange(betaMultChange)
sch3.sortChanges()
sch3.simulate()

In [60]:
curRes = sch3.result 
RTs = 1 - curRes.y.sum(axis=0)

fig,(ax1,ax2) = plt.subplots(2,1)
curLabels = [   
                r'$Y$: Infectious',
                r'$Y_{1,0}$: Infectious'+'\n(previous I)',
                r'$I_{0,1}$: Infectious'+'\n(previous Y)',
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                ]
curColors = ['xkcd:red','xkcd:light red','xkcd:yellow','xkcd:orange','xkcd:light orange']
                
for ax in (ax1,ax2):
    ax.stackplot(curRes.t,
                DKpop*curRes.Y,
                DKpop*curRes.Y10,
                DKpop*curRes.I01,
                DKpop*curRes.IS,
                DKpop*curRes.IV,
                labels=curLabels,colors=curColors)

t_width = 30*5
t_init = 90

ax1.legend()

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=8000)
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax2.set_xlim([t_init,t_init+t_width])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(90.0, 240.0)

# Try with some partial immunity, reflected by lower betas

In [73]:
tChange2 = t_init + 75 # Approximately December 15th
betaMult = 1
betaMult_Y = betaMult *0.6
betaMult_I = betaMult
betaMult_I_V = betaMult * 0.6
betaMult_I_R = betaMult * 0.6
# betaMult = 0.7
# betaMult_Y = betaMult *0.8
# betaMult_I = betaMult 
dictToMult = {
'beta_IS_S' : betaMult_I,
'beta_IV_S' : betaMult_I,
'beta_I01_S' : betaMult_I,
'beta_IS_V' : betaMult_I_V,
'beta_IV_V' : betaMult_I_V,
'beta_I01_V' : betaMult_I_V,
'beta_IS_R01' : betaMult_I_R,
'beta_IV_R01' : betaMult_I_R,
'beta_I01_R01' : betaMult_I_R,
'beta_Y_S' : betaMult_Y,
'beta_Y10_S' : betaMult_Y,
'beta_Y_R10' : betaMult_Y,
'beta_Y10_R10' : betaMult_Y,
'gamma_IS' : 1,
'gamma_IV' : 1,
'gamma_Y' : 1,
'gamma_I01' : 1,
'gamma_Y10' : 1,
}
betaMultChange = pmf.Change(tChange2, MultiplyParameters=dictToMult)

sch4 = sch2.copy()
sch4.addChange(betaMultChange)
sch4.sortChanges()
sch4.simulate()

In [74]:
curRes = sch4.result 
RTs = 1 - curRes.y.sum(axis=0)

fig,(ax1,ax2) = plt.subplots(2,1)
curLabels = [   
                r'$Y$: Infectious',
                r'$Y_{1,0}$: Infectious'+'\n(previous I)',
                r'$I_{0,1}$: Infectious'+'\n(previous Y)',
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                ]
curColors = ['xkcd:red','xkcd:light red','xkcd:yellow','xkcd:orange','xkcd:light orange']
                
for ax in (ax1,ax2):
    ax.stackplot(curRes.t,
                DKpop*curRes.Y,
                DKpop*curRes.Y10,
                DKpop*curRes.I01,
                DKpop*curRes.IS,
                DKpop*curRes.IV,
                labels=curLabels,colors=curColors)

t_width = 30*5
t_init = 90

ax1.legend()

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0,top=8000)
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax2.set_xlim([t_init,t_init+t_width])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(90.0, 240.0)

In [75]:
curRes = sch4.result 
RTs = 1 - curRes.y.sum(axis=0)

fig,ax1 = plt.subplots()
curLabels = [   
                r'$I_S$: Infectious'+'\n(unvaccinated)',
                r'$I_V$: Infectious'+'\n(vaccinated)',
                r'$I_{0,1}$: Infectious'+'\n(previous Y)',
                r'$Y$: Infectious',
                r'$Y_{1,0}$: Infectious'+'\n(previous I)',
                r'$R_{1,0}$: Recovered, I',
                r'$R_{0,1}$: Recovered, Y',
                r'$R_{1,1}$: Immune to both',
                r'$S$: Susceptible',
                r'$V$: Vaccinated'
                ]
curColors = ['xkcd:orange','xkcd:light orange','xkcd:yellow','xkcd:red','xkcd:light red','xkcd:green','xkcd:light green','xkcd:dark green','xkcd:sky blue','xkcd:grey']
ax1.stackplot(curRes.t,
                curRes.IS,
                curRes.IV,
                curRes.I01,
                curRes.Y,
                curRes.Y10,
                curRes.R10,
                curRes.R01,
                RTs,
                curRes.S,
                curRes.V,
                labels=curLabels,colors=curColors)

ax1.legend(bbox_to_anchor=(1,0.5,0.25,0.1),loc='center right',mode='expand') 
ax1.set_ylim([0,1])
# ax1.set_ylim([0,0.25])
ax1.set_xlim([curRes.t[0],curRes.t[-1]])
ax1.set_xlabel('t')
ax1.set_ylabel('Share of population')
fig.tight_layout()

# fig.savefig(path_figs+'IthenY')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …